In [2]:
%load_ext sql
%sql postgresql+psycopg://db:db@postgres/db

There's a new jupysql version available (0.10.4), you're running 0.10.1. To upgrade: pip install jupysql --upgrade


In [ ]:
%%sql

%%sql

DROP TABLE IF EXISTS prescribes;
DROP TABLE IF EXISTS Medicine;
DROP TABLE IF EXISTS Tooth;
DROP TABLE IF EXISTS DentalCharting;
DROP TABLE IF EXISTS involves;
DROP TABLE IF EXISTS Procedure_;
DROP TABLE IF EXISTS relates;
DROP TABLE IF EXISTS results;
DROP TABLE IF EXISTS Diagnosis;
DROP TABLE IF EXISTS Consultation;
DROP TABLE IF EXISTS Report;
DROP TABLE IF EXISTS Trainee;
DROP TABLE IF EXISTS Supervisor;
DROP TABLE IF EXISTS Permanent;
DROP TABLE IF EXISTS Appointment;
DROP TABLE IF EXISTS Doctor;
DROP TABLE IF EXISTS Nurse;
DROP TABLE IF EXISTS Receptionist;
DROP TABLE IF EXISTS Client;
DROP TABLE IF EXISTS Employee;
DROP TABLE IF EXISTS Person;
DROP TABLE IF EXISTS Address;

CREATE TABLE Address (
    street VARCHAR(255),
    zip VARCHAR(12),
    city VARCHAR(30),
    PRIMARY KEY(street, zip, city)
    -- Every Address must exist in the table 'Person' 
);

CREATE TABLE Person (
    vat VARCHAR(20),
    name VARCHAR(80) NOT NULL,
    dob DATE NOT NULL,
    phone NUMERIC(9) NOT NULL,
    street VARCHAR(255),
    zip VARCHAR(12),
    city VARCHAR(30),
    PRIMARY KEY(vat),
    FOREIGN KEY(street, zip, city) REFERENCES Address(street, zip, city),
    UNIQUE (phone)
    -- Every Person must exist either in the table 'Client' or in the table 'Employee'
);

CREATE TABLE Employee (
    vat VARCHAR(20),
    salary NUMERIC (16,4) NOT NULL,
    iban VARCHAR(30) NOT NULL,
    PRIMARY KEY(vat),
    FOREIGN KEY(vat) REFERENCES Person(vat),
    UNIQUE (iban)
    -- No Employee can exist at the same time in the both the table 'Nurse' or in the table 'Doctor'
    -- No Employee can exist at the same time in the both the table 'Receptionist' or in the table 'Doctor'
    -- No Employee can exist at the same time in the both the table 'Nurse' or in the table 'Receptionist'
    -- Every Employee must exist either in the table 'Nurse' or in the table 'Doctor' or in the table 'Receptionist' 
);

CREATE TABLE Client (
    vat VARCHAR(20),
    gender CHAR(1) NOT NULL,
    PRIMARY KEY(vat),
    FOREIGN KEY(vat) REFERENCES Person(vat)
);

CREATE TABLE Receptionist (
    vat VARCHAR(20),
    PRIMARY KEY(vat),
    FOREIGN KEY(vat) REFERENCES Employee(vat)
);

CREATE TABLE Nurse (
    vat VARCHAR(20),
    PRIMARY KEY(vat),
    FOREIGN KEY(vat) REFERENCES Employee(vat)
);

CREATE TABLE Doctor (
    vat VARCHAR(20),
    email VARCHAR(255) NOT NULL,
    specialization VARCHAR(200) NOT NULL,
    bio TEXT NOT NULL, 
    PRIMARY KEY(vat),
    FOREIGN KEY(vat) REFERENCES Employee(vat),
    UNIQUE(email)
    -- No Employee can exist at the same time in the both the table 'Permanent' or in the table 'Trainee'
    -- Every Employee must exist either in the table 'Permanent' or in the table 'Trainee'
);

CREATE TABLE Permanent (
    vat VARCHAR(20),
    start_practice DATE NOT NULL,
    PRIMARY KEY(vat),
    FOREIGN KEY(vat) REFERENCES Doctor(vat)
);

CREATE TABLE Supervisor (
    vat VARCHAR(20),
    PRIMARY KEY(vat),
    FOREIGN KEY(vat) REFERENCES Permanent(vat)
    -- Every Supervisor must exist in the table 'Trainee' 
);

CREATE TABLE Trainee (
    vat VARCHAR(20),
    supervisor_vat VARCHAR(20),
    PRIMARY KEY(vat),
    FOREIGN KEY(vat) REFERENCES Doctor(vat),
    FOREIGN KEY(supervisor_vat) REFERENCES Supervisor(vat)
);

CREATE TABLE Report(
    content TEXT NOT NULL,
    date DATE,
    score NUMERIC (3,2) NOT NULL, 
    trainee_vat VARCHAR(20),
    supervisor_vat VARCHAR(20),
        
    PRIMARY KEY(date, trainee_vat),
    FOREIGN KEY(trainee_vat) REFERENCES Trainee(vat),
    FOREIGN KEY(supervisor_vat) REFERENCES Supervisor(vat)
    
    -- Every Reports regarding the same Supervisor and Trainee must have dates at least 2 months apart.
    -- Every Report score must be in the range from 1 to 5.
    -- #TODO IC4: confirmem! reescrever no draw.io?
    -- #ir buscar supervisor_vat à table Trainee em vez de à table Supervisor garante esta constraint? 
    --Every Report must have a Supervisor and a Trainee, in which 
    --the Trainee has that same Supervisor in the table 'Trainee'

);

CREATE TABLE Appointment(
    description TEXT NOT NULL,
    date TIMESTAMP,
    doctor_vat VARCHAR(20),
    client_vat VARCHAR(20),
    receptionist_vat VARCHAR(20),
    
    PRIMARY KEY(date, doctor_vat),
    FOREIGN KEY(doctor_vat) REFERENCES Doctor(vat),
    FOREIGN KEY(client_vat) REFERENCES Client(vat),
    FOREIGN KEY(receptionist_vat) REFERENCES Receptionist(vat)
);

CREATE TABLE Consultation (
    subjective TEXT NOT NULL,
    objective TEXT NOT NULL,
    assessment TEXT NOT NULL,
    plan TEXT NOT NULL,
    doctor_vat VARCHAR(20),
    date DATE,
    PRIMARY KEY(date, doctor_vat),
    FOREIGN KEY(date, doctor_vat) REFERENCES Appointment(date, doctor_vat)
);

CREATE TABLE Diagnosis (
    description TEXT NOT NULL,
    code VARCHAR(200), --# TODO Max characters in SNODENT? 
    PRIMARY KEY(code)
    -- Every Diagnosis code must be standardized.
);

CREATE TABLE results (
    doctor_vat VARCHAR(20),
    date DATE,
    code VARCHAR(200), --# TODO Max characters in SNODENT? 
    PRIMARY KEY(date, doctor_vat, code),
    FOREIGN KEY(code) REFERENCES Diagnosis(code),
    FOREIGN KEY(date, doctor_vat) REFERENCES Consultation(date, doctor_vat)
);

CREATE TABLE relates (
    code_1 VARCHAR(200), --# TODO Max characters in SNODENT? 
    code_2 VARCHAR(200), --# TODO Max characters in SNODENT? 
    relation_type VARCHAR(80) NOT NULL,
    PRIMARY KEY(code_1, code_2),
    FOREIGN KEY(code_1) REFERENCES Diagnosis(code),
    FOREIGN KEY(code_2) REFERENCES Diagnosis(code)
    -- Every relates code_1 and code_2 must be different
);

CREATE TABLE Procedure_ (
    name VARCHAR(80), --#TODO Size?
    type VARCHAR(150), --#TODO Size?
    PRIMARY KEY(name, type)
    -- Every Procedure_ must exist in the table 'involves'
    -- Every Procedure_ with type equal to "Dental Charting" must exist in the table 'DentalCharting'
);

CREATE TABLE involves (
    doctor_vat VARCHAR(20),
    date DATE,
    name VARCHAR(80), --#TODO Size?
    type VARCHAR(150), --#TODO Size?
    text_description TEXT NOT NULL,
    PRIMARY KEY(date, doctor_vat, name, type),
    FOREIGN KEY(date, doctor_vat) REFERENCES Consultation(date, doctor_vat),
    FOREIGN KEY(name, type) REFERENCES Procedure_(name, type)
);

CREATE TABLE DentalCharting (
    name VARCHAR(80), --#TODO Size?
    type VARCHAR(150), --#TODO Size?
    PRIMARY KEY(name, type),
    FOREIGN KEY(name, type) REFERENCES Procedure_(name, type)
);

CREATE TABLE Tooth(
    mouth_quad CHAR(1),
    gap CHAR(1) NOT NULL, --#TODO confirmem size 1
    number NUMERIC (16,4),
    name VARCHAR(80), --#TODO Size?
    type VARCHAR(150), --#TODO Size?    
    PRIMARY KEY(name, type, mouth_quad, number),
    FOREIGN KEY(name, type) REFERENCES DentalCharting(name, type)
    --Every Tooth gap must be in milimeters
    --Every Tooth mouth_quad and number must be according to the Palmer Notation Numbering System
);

CREATE TABLE Medicine (
    laboratory VARCHAR(80),
    active_sub VARCHAR(80),
    PRIMARY KEY(laboratory, active_sub)
);

CREATE TABLE prescribes (
    laboratory VARCHAR(80),
    active_sub VARCHAR(80),
    doctor_vat VARCHAR(20),
    date DATE,
    code VARCHAR(200), --# TODO Max characters in SNODENT? 
    dosage VARCHAR(80) NOT NULL,
    dosage_regime TEXT NOT NULL,
    PRIMARY KEY(laboratory, active_sub, date, doctor_vat, code),
    FOREIGN KEY(laboratory, active_sub) REFERENCES Medicine(laboratory, active_sub),
    FOREIGN KEY(date, doctor_vat, code) REFERENCES results(date, doctor_vat, code)
);

: 